In [28]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# File name: VLBI_internal_error2.py
"""
Created on Thu Jun 21 09:30:04 2018

@author: Neo(liuniu@smail.nju.edu.cn)
"""

import matplotlib.pyplot as plt
import numpy as np
from numpy import cos, deg2rad, sqrt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# My modules
from error_inflation_progs import (error_inflation, pos_offset_wrms_soubinned, wrms_calc)

In [29]:
# Load data
datfile = ("../data/odd-even-b.cat_diff")
soulist = np.genfromtxt(datfile, usecols=(0,), dtype=str)
RA, Dec, dRAc, dDec, dRAc_err, dDec_err = np.genfromtxt(
    datfile, usecols=range(1, 7), unpack=True)
num_ses_mean, num_obs_mean = np.genfromtxt(
    datfile, dtype=int, usecols=range(7, 9), unpack=True)

In [30]:
# scale the position offset by their uncertainty
nor_dRAc = dRAc / dRAc_err
nor_dDec = dDec / dDec_err

# Calculate the offset and normalized seperation
rho = sqrt(dRAc**2 + dDec**2)
nor_rho = sqrt(nor_dRAc**2 + nor_dDec**2)
rho_err = sqrt(dRAc_err**2 + dDec_err**2)

In [31]:
# Sort the data according to the number of sessions/observations
ind = np.argsort(num_ses_mean)
num_ses_sort = np.take(num_ses_mean, ind)
num_obs_sort = np.take(num_obs_mean, ind)
dRAc_sort = np.take(dRAc, ind)
dDec_sort = np.take(dDec, ind)
dRAc_err_sort = np.take(dRAc_err, ind)
dDec_err_sort = np.take(dDec_err, ind)

# Calculate the positional offset scatter
dRAc_wrms, dRAc_err_med, ses_min_RA = pos_offset_wrms_soubinned(
    num_ses_sort, dRAc_sort, dRAc_err_sort, interv_size=50)
dDec_wrms, dDec_err_med, ses_min_Dec = pos_offset_wrms_soubinned(
    num_ses_sort, dDec_sort, dDec_err_sort, interv_size=50)

In [32]:
# If we use all sources
print("If we choose all %d sources." % nor_dRAc.size)

# Calculate their standard deviation (scale factor)
sf_ra = np.std(nor_dRAc)
m_ra = np.mean(nor_dRAc)
sf_dec = np.std(nor_dDec)
m_dec = np.mean(nor_dDec)

print("For all sources:\n"
      "     scale factors of RA and Dec. are %.3f and %.3f." % (sf_ra, sf_dec))
print("     mean values are %.3f and %.3f" % (m_ra, m_dec))

# Noise floor for post-elimination sources
nf_ra = wrms_calc(dRAc, dRAc_err) / sqrt(2)
nf_dec = wrms_calc(dDec, dDec_err) / sqrt(2)

print("For all sources:\n"
      "     noise floors of RA and Dec. are %.3f and %.3f mas." % (nf_ra, nf_dec))
print("     means are %.3f and %.3f mas." % (np.mean(dRAc), np.mean(dDec)))

# Inflated error
dRAc_err_inf = error_inflation(dRAc_err_med, sf_ra, nf_ra)
dDec_err_inf = error_inflation(dDec_err_med, sf_dec, nf_dec)

If we choose all 2380 sources.
For all sources:
     scale factors of RA and Dec. are 1.305 and 1.247.
     mean values are -0.020 and 0.028
For all sources:
     noise floors of RA and Dec. are 0.033 and 0.040 mas.
     means are 0.048 and -0.131 mas.


In [33]:
# If we use a criteria "offset <= 1 mas" and "normalized seperation < 5"
# con = (rho <= 1.) & (nor_rho <= 5.)
con = (rho_err <= 1.) & (rho/rho_err <= 3.97)

print("If we choose 'offset <= 1 mas' as the criteria, there will be %d sources." % nor_dRAc[con].size)

# Calculate their standard deviation (scale factor)
sf_ra_elim1 = np.std(nor_dRAc[con])
m_ra_elim1 = np.mean(nor_dRAc[con])
sf_dec_elim1 = np.std(nor_dDec[con])
m_dec_elim1 = np.mean(nor_dDec[con])

print("For sources (offset < 1 mas and normalized offset < 5):\n"
      "             scale factors of RA and Dec. are %.3f and %.3f." % (sf_ra_elim1, sf_dec_elim1))
print("             mean values are %.3f and %.3f" % (m_ra_elim1, m_dec_elim1))

# Noise floor for post-elimination sources
nf_ra_elim1 = wrms_calc(dRAc[con], dRAc_err[con]) / sqrt(2)
nf_dec_elim1 = wrms_calc(dDec[con], dDec_err[con]) / sqrt(2)

print("For sources (offset < 1 mas and normalized offset < 5):\n"
      "             noise floors of RA and Dec. are %.3f and %.3f mas." % (nf_ra_elim1, nf_dec_elim1))
print("             means are %.3f and %.3f mas." % (np.mean(dRAc[con]), np.mean(dDec[con])))

# Inflated error
dRAc_err_inf_elim1 = error_inflation(dRAc_err_med, sf_ra_elim1, nf_ra_elim1)
dDec_err_inf_elim1 = error_inflation(dDec_err_med, sf_dec_elim1, nf_dec_elim1)

If we choose 'offset <= 1 mas' as the criteria, there will be 1756 sources.
For sources (offset < 1 mas and normalized offset < 5):
             scale factors of RA and Dec. are 1.342 and 1.274.
             mean values are -0.036 and 0.047
For sources (offset < 1 mas and normalized offset < 5):
             noise floors of RA and Dec. are 0.029 and 0.035 mas.
             means are -0.008 and 0.014 mas.


In [34]:
# If we use a criteria "offset <= 0.5 mas"
# con = (rho <= 0.5) & (nor_rho <= 5.)
con = (rho <= 0.5) & (rho/rho_err <= 5.)

print("If we choose 'offset <= 0.5 mas' as the criteria, there will be %d sources." % nor_dRAc[con].size)

# Calculate their standard deviation (scale factor)
sf_ra_elim2 = np.std(nor_dRAc[con])
m_ra_elim2 = np.mean(nor_dRAc[con])

sf_dec_elim2 = np.std(nor_dDec[con])
m_dec_elim2 = np.mean(nor_dDec[con])

print("For sources (offset < 0.5 mas and normalized offset < 5):\n"
      "             scale factors of RA and Dec. are %.3f and %.3f." % (sf_ra_elim2, sf_dec_elim2))
print("             mean values are %.3f and %.3f" % (m_ra_elim2, m_dec_elim2))

# Noise floor for post-elimination sources
nf_ra_elim2 = wrms_calc(dRAc[con], dRAc_err[con]) / sqrt(2)
nf_dec_elim2 = wrms_calc(dDec[con], dDec_err[con]) / sqrt(2)

print("For sources (offset < 0.5 mas and normalized offset < 5):\n"
      "             noise floors of RA and Dec. are %.3f and %.3f mas." % (nf_ra_elim2, nf_dec_elim2))
print("             means are %.3f and %.3f mas." % (np.mean(dRAc[con]), np.mean(dDec[con])))

# Inflated error
dRAc_err_inf_elim2 = error_inflation(dRAc_err_med, sf_ra_elim2, nf_ra_elim2)
dDec_err_inf_elim2 = error_inflation(dDec_err_med, sf_dec_elim2, nf_dec_elim2)

If we choose 'offset <= 0.5 mas' as the criteria, there will be 1287 sources.
For sources (offset < 0.5 mas and normalized offset < 5):
             scale factors of RA and Dec. are 1.217 and 1.073.
             mean values are -0.020 and 0.032
For sources (offset < 0.5 mas and normalized offset < 5):
             noise floors of RA and Dec. are 0.023 and 0.025 mas.
             means are -0.004 and 0.001 mas.


In [35]:
# If we use a criteria "offset <= 0.1 mas"
# con = (rho <= 0.1) & (nor_rho <= 5.)
con = (rho <= 0.1) & (rho/rho_err <= 5.)

print("If we choose 'offset <= 0.1 mas' as the criteria, there will be %d sources." % nor_dRAc[con].size)

# Calculate their standard deviation (scale factor)
sf_ra_elim3 = np.std(nor_dRAc[con])
m_ra_elim3 = np.mean(nor_dRAc[con])

sf_dec_elim3 = np.std(nor_dDec[con])
m_dec_elim3 = np.mean(nor_dDec[con])

print("For sources (offset < 0.1 mas and normalized offset < 5):\n"
      "             scale factors of RA and Dec. are %.3f and %.3f." % (sf_ra_elim3, sf_dec_elim3))
print("             mean values are %.3f and %.3f" % (m_ra_elim3, m_dec_elim3))

# Noise floor for post-elimination sources
nf_ra_elim3 = wrms_calc(dRAc[con], dRAc_err[con]) / np.sqrt(2)
nf_dec_elim3 = wrms_calc(dDec[con], dDec_err[con]) / np.sqrt(2)

print("For sources (offset < 0.5 mas and normalized offset < 5):\n"
      "             noise floors of RA and Dec. are %.3f and %.3f mas." % (nf_ra_elim3, nf_dec_elim3))
print("             means are %.3f and %.3f mas." % (np.mean(dRAc[con]), np.mean(dDec[con])))

# Inflated error
dRAc_err_inf_elim3 = error_inflation(dRAc_err_med, sf_ra_elim3, nf_ra_elim3)
dDec_err_inf_elim3 = error_inflation(dDec_err_med, sf_dec_elim3, nf_dec_elim3)

If we choose 'offset <= 0.1 mas' as the criteria, there will be 444 sources.
For sources (offset < 0.1 mas and normalized offset < 5):
             scale factors of RA and Dec. are 1.194 and 1.135.
             mean values are -0.030 and 0.169
For sources (offset < 0.5 mas and normalized offset < 5):
             noise floors of RA and Dec. are 0.013 and 0.016 mas.
             means are -0.001 and 0.004 mas.


In [36]:
# Summary
# a criteria "offset <= 1 mas" and "normalized seperation < 5"
# print("For sources (offset < 1 mas and normalized offset < 3.97):\n"
print("For sources (normalized offset X < 3.97 and positional difference error < 1 mas):\n"
      "             scale factors of RA and Dec. are %.3f and %.3f.\n" % (sf_ra_elim1, sf_dec_elim1),
      "             noise floors of RA and Dec. are %.3f and %.3f mas." % (nf_ra_elim1, nf_dec_elim1))

# a criteria "offset <= 0.5 mas" and "normalized seperation < 5"
print("For sources (offset <= 0.5 mas and normalized offset < 5):\n"
      "             scale factors of RA and Dec. are %.3f and %.3f.\n" % (sf_ra_elim2, sf_dec_elim2),
      "             noise floors of RA and Dec. are %.3f and %.3f mas." % (nf_ra_elim2, nf_dec_elim2))

# a criteria "offset <= 0.1 mas" and "normalized seperation < 5"
print("For sources (offset <= 0.1 mas and normalized offset < 5):\n"
      "             scale factors of RA and Dec. are %.3f and %.3f.\n" % (sf_ra_elim3, sf_dec_elim3),
      "             noise floors of RA and Dec. are %.3f and %.3f mas." % (nf_ra_elim3, nf_dec_elim3))

For sources (normalized offset X < 3.97):
             scale factors of RA and Dec. are 1.342 and 1.274.
              noise floors of RA and Dec. are 0.029 and 0.035 mas.
For sources (offset <= 0.5 mas and normalized offset < 5):
             scale factors of RA and Dec. are 1.217 and 1.073.
              noise floors of RA and Dec. are 0.023 and 0.025 mas.
For sources (offset <= 0.1 mas and normalized offset < 5):
             scale factors of RA and Dec. are 1.194 and 1.135.
              noise floors of RA and Dec. are 0.013 and 0.016 mas.
